# Aim of This Notebook

### THEORY REFERENCE: Spitzer Identity, Wiener-Hopf Factorization and Pricing of Discretely Monitored Exotic Options by Fusai, Germano, Marazzina (May 18, 2016). European Journal of Operational Research.

The aim of this notebook is to provide an example of usage of convolution methodologies for plain vanilla option pricing. 

> Advantages: semi-closed formula ($\to$fast), supports jumps diffusion or general Lévy processes (here just GBM and NIG are presented), supports barrier options (here Down and Out is presented).

> Disadvantages: suitable only for plain vanilla and barrier options (at the best of my knowledge).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftshift, ifftshift
from scipy.interpolate import interp1d

class DOPricing:
    def __init__(self, parameters):
        self.parameters = parameters
        self.parameters['dt'] = self.parameters['T'] / self.parameters['Ndate']

    def charfunction(self, u, flag=0):
        meancorrection = (self.parameters['rf'] - self.parameters['q']) * self.parameters['dt'] - np.log(self.charfunction0(-1j))
        F = np.exp(1j * meancorrection * u) * self.charfunction0(u)
        if flag == 0:
            F = np.conj(F)
        return F

    def charfunction0(self, u):
        dt = self.parameters['dt']
        if self.parameters['distr'] == 'Normal':  # Normal
            m = self.parameters['mu'] * dt
            s = self.parameters['sigma'] * np.sqrt(dt)
            F = np.exp(1j * u * m - 0.5 * (s * u) ** 2)
        elif self.parameters['distr'] == 'Normal Inverse Gaussian':  # Normal inverse Gaussian (NIG)
            alpha = self.parameters['alpha']
            beta = self.parameters['beta']
            delta = self.parameters['delta'] * dt
            F = np.exp(-delta * (np.sqrt(alpha ** 2 - (beta + 1j * u) ** 2) - np.sqrt(alpha ** 2 - beta ** 2)))
        return F

    def kernel(self, ngrid, xmin, xmax, flag=0):
        N = ngrid // 2
        dx = (xmax - xmin) / ngrid
        x = dx * np.arange(-N, N)
        dw = 2 * np.pi / (xmax - xmin)
        w = dw * np.arange(-N, N)
        
        H = self.charfunction(w, flag)
        
        return x, H

    def CONV(self, S_0, K, B, N=2**14):
        b = 2.5
        x, H = self.kernel(N, -b, b)
        S = S_0 * np.exp(x)
        
        v = np.maximum(S - K, 0) * (S > B)
        H = ifftshift(H)
        
        for j in range(self.parameters['Ndate']):
            v = np.real(fftshift(fft(ifft(ifftshift(v)) * H))) * np.exp(-self.parameters['rf'] * self.parameters['dt'])
            v[S <= B] = 0
        
        index = np.where((S > 0.5 * S_0) & (S < 2 * S_0))
        S = S[index]
        v = v[index]
        
        return interp1d(S, v, kind='cubic')(S_0)

In [2]:
# Usage Example
parameters = {
    'rf': 0.02,          # risk-free rate
    'q': 0,              # dividend
    'distr': 'Normal Inverse Gaussian',   # normal distribution
    'alpha': 1,             
    'beta': 0.2,       
    'delta': 0.4,
    'T': 1,              # maturity
    'Ndate': 12          # number of dates
}


option_pricing = DOPricing(parameters)

S_0 = 100
K = 100
B = 90

price = option_pricing.CONV(S_0, K, B)

print("Price:", price)

Price: 9.344582439795257
